## DataCite Metadata Extraction for Digital Commons Batch Upload

### What does this code do?

This code searches using the DataCite API for datasets. It then downloads those metadata records and converts them into a format that can be used for a Digital Commons bulk upload, with appropriate headers. This file will be saved as an .xlsx file and will require manual re-saving to a .xls file for the actual upload. 

### Additional Manual Curation

This file will also require manual curation before it is ready for upload, with special attention to:

- Cleaning up affiliations, ex:
  - Unifying names for a single institution: University of Alabama - Birmingham, UNIVERSITY OF ALABAMA AT BIRMINGHAM
   &rarr; University of Alabama at Birmingham
  - Un-abbreviating institution names: NYU &rarr; New York University 
  - Removing extraneous location details: California Digital Library, Oakland, United States of America &rarr; California Digital Library
  - Ensuring there is only one affiliation per person (Digital Commons currently only allows a single affiliation)
- Checking for names and keywords written in all caps
- Checking for special characters or accents that are not formatted properly

### What datasets are included?

We want to include find datasets where at least one author is affiliated with the University if Alabama at Birmingham. We search requiring that the type of object is "Dataset" and using the UAB ROR in the affiliation-id. DataCite mints DOIs for several repositories, inclusing Dryad, Zenodo, and Figshare. It also may generate DOIs for other repositories.

### Import the data as a json

The code below uses a request url, `url_request`, to search using the DataCite API. To customize this, edit the URL to insert your own institution's. 

We are left with `results`, the API response in json format, which gets saved in the raw-data directory.

In [1]:
import requests
import json
import datetime

ror_url = 'https://ror.org/008s83205' # edit this to change institution

url_request =  'https://api.datacite.org/dois?affiliation-id=' + ror_url + '&resource-type-id=Dataset&affiliation=true&page[size]=1000'

response = requests.get(url_request)
if response.ok:
  results = response.json()

today = str(datetime.date.today())

with open('raw-data/datacite_response' + today + '.json', 'w', encoding='utf-8') as f:
    json.dump(results, f, ensure_ascii=False, indent=4)

print('Results returned from API search:')
print(len(results['data']))

Results returned from API search:
390


### Initial Data Reduction

There are a few initial steps before we get into the proper reformatting of the records.

1. Load the raw data from the json file and convert it to a Pandas dataframe.
2. Save this dataframe as a csv (with the termination "-full") so we can view the raw data in a spreadsheet format.
3. Remove Figshare duplicates. Figshare will often have multiple DOI versions that all lead to the same dataset, and are all individually returned by the DataCite API search. Therefore, we remove any figshare DOI that has a ".v" in it, since the un-versioned original DOI will always point to the most recent version of the dataset.
4. Remove datasets with excessive author lists. Digital Commons can accept up to 33 authors per entry. Any dataset with more than 33 authors is removed, but its DOI is noted in the "long_list_names_[today].txt" file. If Digital Commons changes their policy later, or you choose to include the full author list in another metadata field, you can have access to these datasets.
   
After performing these reductions, we are left with `df_input`, the dataframe we will use when we process and reformat the records. We save this as a .csv for easier inspection.

In [ ]:
import pandas as pd
import json

path = 'raw-data/datacite_response' + today + '.json'

data = results
df = pd.json_normalize(data["data"], max_level = 1)

df.to_csv('raw-data/raw-datacite'+today+'-full.csv', index=False, encoding='utf-8-sig')

# Remove multiple figshare duplicate versions (DOI+.v#)
df = df[~df['id'].str.contains('figshare', case=False, na=False) | 
        ~df['id'].str.contains(r'\.v', na=False)]

# Filter names where the list column has more than 33 elements
names = df.loc[df['attributes.creators'].apply(lambda x: len(x) > 33), 'id']

# keep only under 33 names due to DC limit on author count

df = df[df['attributes.creators'].apply(lambda x: len(x) <= 33)]

# Write to a text file, one name per line
with open('long_list_names_' + today + '.txt', 'w') as f:
    for name in names:
        f.write('https://doi.org/' + f"{name}\n")

df_input = df

df.to_csv('raw-data/raw-datacite-'+today+'.csv', index=False, encoding='utf-8-sig')

## Helper Functions

Functions defined below reshape the data from the initial spreadsheet/dataframe `df_input` and add columns to the dataframe df_output, which will later be exported and uploaded to Digital Commons.

In [ ]:

import csv 

def format_orcid(orcid):
    if orcid[0] != 'h':
        url_orcid = 'https://orcid.org/' + orcid
        num_orcid = orcid
    else:
        url_orcid = orcid
        num_orcid = orcid[18:]
    return url_orcid, num_orcid

def to_html(string):
    '''Takes in a string. If the string is not in html already (assume first character is <) wrap it in <p> ... </p>.'''
    if string[0] != "<":
        string = "<p>" + string + "</p>"
    return(string)

def list_to_string(lst):
    '''Takes in a list of strings ['a', 'b', 'c'] and returns a single string containing the list elements, separated by commas 'a, b, c'. '''
    if isinstance(lst, list):  # Check if the value is a list
        return ', '.join(lst)
    else:
        return ""  # Convert non-list values to string or handle as needed

def csv_to_dict(file_path):
    '''Imports data from a 2-column csv file, where the first column contains dictionary keys and the second column contains the corresponding values.
    Outputs the resulting dictionary. Used with the relation_types.csv file to generate the strings used for the relation type of a related item.'''
    result_dict = {}
    with open(file_path, mode='r', newline='', encoding='utf-8') as csvfile:
        csvreader = csv.reader(csvfile)
        for row in csvreader:
            key = row[0]  # First column as key
            value = row[1]  # Second column as value
            result_dict[key] = value
    return result_dict

def get_titles(df1):
    '''Generates titles column.'''
    titles = []
    for title_dict in df1['attributes.titles']:
        title =  title_dict[0]['title']
        titles.append(title)
    df2 = pd.DataFrame(titles, columns = ['title'])
    return df2   

def add_orcid(df1, df2):
    '''Finds authors with ORCIDs and lists them in html format for each dataset, along with hyperlinked urls.'''
    orcid_pairs = [] # List for lists of author/orcid pairs where each element corresponds to a different dataset 
    for index, row in df1.iterrows():
        pairs = [] # List for appending author/orcid pairs within one dataset
        for author in row['attributes.creators']: # Iterate through the list of authors in each row of the metadata.creators column
            #author = row['attributes.creators'][i]
            if 'nameIdentifiers' in author and len(author['nameIdentifiers']) > 0:  # Check if ORCID is present
                #orcid_url = make_url(author['nameIdentifiers'][0]['nameIdentifier'])
                orcid_url, orcid_num = format_orcid(author['nameIdentifiers'][0]['nameIdentifier'])
                first_last_name = author['givenName'] + ' ' + author['familyName']
                #pairs.append(orcid)
                #first_last_name = reformat_name(author['name'])
                pairs.append('<p>' + first_last_name + ' <a href="' + orcid_url + '">' + orcid_num + '</a></p>')
        orcid_pairs.append("".join(pairs))  # Join multiple name-ORCID pairs with a comma
    # Add this list as a new column in df2
    df2['orcid'] = orcid_pairs
    return df2    

def add_pub_date(df1, df2):
    '''Adds publication date column. Trims date to yyyy-mm-dd format. If only yyy exists, it uses that.'''
    dates = []
    for date_list in df1['attributes.dates']:
        date_types = []
        for date_type in date_list:
            temp_date = date_type['date']

            if len(temp_date) > 4:
                date_types.append(temp_date[:10])
        if len(date_types) == 0:
            date_types.append(date_type['date'])
        date = min(date_types)
        dates.append(date)
    df2['publication_date'] = dates
    return df2

def add_abstract(df1, df2):
    '''Adds abstract column formatted in html.'''
    abstracts = []
    for description_dict in df1['attributes.descriptions']:
        if len(description_dict) > 0:
            abstract = to_html(description_dict[0]['description'])
            abstracts.append(abstract)
        else:
            abstracts.append('')
    df2['abstract'] = abstracts
    return df2

def add_keywords(df1, df2):
    '''Adds keywords column in the format of a comma separated list'''
    keywords = []
    for keyword_list in df1['attributes.subjects']:
        keyword_sublist = []
        for word in keyword_list:
            subject = word['subject']
            keyword_sublist.append(subject)
        keywords.append(list_to_string(keyword_sublist))
    df2['keywords'] = keywords
    return df2

relation_dict = csv_to_dict('relation_types_datacite.csv')

def add_related_items(df1, df2):
    '''Adds related items from df1 to df2 with html formatting. Includes relation type taken from relation_dict dictionary. Formats PID appropriately if it is a DOI or other URL.'''
    items = []
    for index, row in df1.iterrows():
        item = []
        # Iterate through the related items if they are present
        if isinstance(row['attributes.relatedIdentifiers'], list):
            item.append('<p>')
            count =0
            for id in row['attributes.relatedIdentifiers']:
                if pd.notnull(id):  
                    if count > 0:
                        item.append('<br>')
                    count += 1
                    if id['relationType']:
                        item.append(relation_dict[id['relationType']] + ': ')
                    if id['relatedIdentifierType'] == 'URL':
                        url = id['relatedIdentifier']
                        item.append( '<a href="'+ url + '">' + url + '</a>')
                    if id['relatedIdentifierType'] == 'DOI':
                        doi = id['relatedIdentifier']
                        item.append('<a href="https://doi.org/' + doi + '">' + doi + '</a>')
                    if id['relatedIdentifierType'] != 'URL' and id['relatedIdentifierType'] != 'DOI':
                        item.append(id['relatedIdentifier'])
            item.append('</p>')
        else:
            # If it's not a list put an empty cell
            if pd.notnull(row['attributes.relatedIdentifiers']):
                item.append('')
        # Append the joined funder information to the list
        items.append("".join(item))
    # Add the new 'fundref' column to df2
    df2['related_data'] = items
    return df2

def add_funders(df1, df2):
    '''Adds funder information from df1 to df2. Adds funder name, and optionally DOI, grant title, and grant number.'''
    funders = []
    for index, row in df1.iterrows():
        funder = []
        # Iterate through the grants if they are present
        if isinstance(row['attributes.fundingReferences'], list):
            for grant in row['attributes.fundingReferences']:
                if pd.notnull(grant):  # Check if grant is not null
                    funder.append('<p>Funder: ' + grant['funderName'])
                    if 'funderIdentifierType' in grant and grant['funderIdentifierType'] == 'Crossref Funder ID':
                        funder.append('<br>Funder DOI: <a href="https://doi.org/' + grant['funderIdentifier'] + '">' +grant['funderIdentifier']+ '</a>')
                    if 'funderIdentifierType' in grant and grant['funderIdentifierType'] == 'ROR':
                        funder.append('<br>Funder ROR: <a href="' + grant['funderIdentifier'] + '">' + grant['funderIdentifier'] + '</a>')
                    if 'awardTitle' in grant:
                        funder.append('<br>' + str(grant['awardTitle']))
                    if 'awardNumber' in grant:
                        funder.append('<br>' + grant['awardNumber'])
                    funder.append('</p>')
        else:
            # If it's not a list put an empty cell
            if pd.notnull(row['metadata.grants']):
                funder.append('')
        # Append the joined funder information to the list
        funders.append("".join(funder))
    # Add the new 'fundref' column to df2
    df2['fundref'] = funders
    return df2



def add_repository(df1, df2):
    '''Adds publisher as repository. Note that for the figshare items added directly by publishers, the publisher will appear.
    This is intentional so that these records may be inspected.'''
    repositories = []
    for repository in df1['attributes.publisher']:
        repositories.append('<p>' + repository + '</p>')
    df2['external_rep'] = repositories
    return df2

### Dictionary containing the DataCite terms for licenses (rightsIdentifier) as keys, with the values being a list containing the license URL as well as the html formatted text corresponding to each license.
license_dict = {"mit" : ["http://opensource.org/license/mit", "<p>This data is available under the MIT License</p>"],
                "cc0-1.0" : ["http://creativecommons.org/public-domain/cc0/", "<p>This data is public domain under the CC-0.0 License</p>"],
                "cc-by-4.0" : ["http://creativecommons.org/licenses/by/4.0/", "<p>This data is available under the CC-BY 4.0 License</p>"],
                "cc-by-nc-nd-4.0" : ["http://creativecommons.org/licenses/by-nc-nd/4.0/", "<p>This data is available under the CC BY-NC-ND 4.0 License</p>"],
                "cc-by-2.0" : ["http://creativecommons.org/licenses/by/2.0/", "<p>This data is available under the CC-BY 2.0 License</p>"],
                "cc-by" : ["http://creativecommons.org/licenses/by/1.0/", "<p>This data is available under the CC-BY License</p>"],
                "" : ["", "<p>Access to this data is restricted.</p>"]
}

def add_license(df1, df2):
    '''Adds licensing information columns to df2. Finds the value of the column metadata.license.id in df1
    and uses it as a key for license_dict to retrieve the url fo the license and the string we want displayed in html.
    If there is no value in that row, there is no license shown. We assume this means the data is restricted.'''
    ids = []
    for license_list in df1['attributes.rightsList']:
        if len(license_list) > 0:
            id = license_list[0]['rightsIdentifier']
        else:
            id = ''
        ids.append(id)
    licenses = [] # List for the license url
    access = [] # List for the string/text explaining access
    for id in ids:
        #print(license_dict[id][0])
        licenses.append(str(license_dict[id][0]))
        access.append(str(license_dict[id][1]))
    df2['distribution_license'] = licenses
    df2['access_link'] = access
    return df2

def doi_link(df1, df2):
    doi_links = []
    for doi in df1['attributes.doi']:
        doi_link = 'https://doi.org/' + doi
        doi_links.append(doi_link)
    df2['source_fulltext_url'] = doi_links
    return df2

def separate_name(name):
    '''Separates a name into First, Middle (if applicable) and Last, returns these elements as separate strings.'''
    middle = ""
    
    # Check if the name contains a comma (indicating "last, first" or "last, first m." format)
    if ',' in name:
        parts = name.split(", ") # Split into a list: "last, first" becomes ["last", "first"]
        last = parts[0] # Last name is everything before the comma
        
        # Check if there's an element after the comma
        if len(parts) > 1:
            first_and_middle = parts[1].split() # Split whatever was after the comma with spaces
            first = first_and_middle[0] # First name will be the first part of that
            
            # Assign middle if available
            if len(first_and_middle) > 1: # If there is a second part
                middle = first_and_middle[1].replace(".", "") # Assign middle name as second part, remove . because DC will add it
        else:
            # Set first to an empty string if there's nothing after the comma
            first = ""
    else: # If the name had no comma, you can assume it is in First Last or First Middle Last format
        parts = name.split() # Split into a list where spaces are 
        
        first = parts[0] # First name is first element of list
        last = parts[-1] # Last name is the last element of the list 
        
        # Check if there is a middle name/initial 
        if len(parts) > 2: 
            middle = parts[1].replace(".", "") # Remove period if it exists
    
    return first, middle, last

def add_creators(df1, df2):
    '''Adds creator information for author lists.
    This function will make a new dataframe df3 and append it on to df2.'''
    # Create a list to hold all rows of data for the new DataFrame
    expanded_data = []
    
    # Process each row in df1
    for _, row in df1.iterrows():
        row_data = {}
        creators = row['attributes.creators']
        
        # Populate the row_data dictionary with each author's name and affiliation
        for i, creator in enumerate(creators):
            author_index = i + 1
            name = creator.get('name', "")
            institution_info = creator.get('affiliation', "")  # Use 'institution' instead of 'affiliation' (DC nomenclature)
            institutions = []
            for inst in institution_info:
                institutions.append(inst['name'])
            
            # Use the separate_name function to split names
            first_name, middle_name, last_name = separate_name(name)
            
            # Assign names and institution to the row_data dictionary
            row_data[f'author{author_index}_fname'] = first_name
            row_data[f'author{author_index}_mname'] = middle_name
            row_data[f'author{author_index}_lname'] = last_name
            row_data[f'author{author_index}_institution'] = list_to_string(institutions)  # Change 'affl' to 'institution'
        
        # Append row_data dictionary to expanded_data list
        expanded_data.append(row_data)

    # Convert expanded_data list of dictionaries into a new DataFrame
    df3 = pd.DataFrame(expanded_data)
    
    # Fill missing values with empty strings for any columns where data is missing
    df3 = df3.fillna("")
    
    df2_reset = df2.reset_index(drop=True)
    df3_reset = df3.reset_index(drop=True)
    
    # Concatenate the two DataFrames along the columns
    df_out = pd.concat([df2_reset, df3_reset], axis=1)
    
    return df_out





## Build output dataframe

Calls the helper functions to build df_output, the dataframe which will them be exported as an Excel sheet.

At this stage, de-duplication is performed. Sometimes, there will be multiple datasets with different DOIs but the same title - we assume these are different versions of the same dataset and keep only the most recent one.

In [74]:
# title
df_output = get_titles(df_input)

# orcid
df_output = add_orcid(df_input, df_output)

# publication_date
df_output = add_pub_date(df_input, df_output)

# abstract
df_output = add_abstract(df_input, df_output)

# keywords
df_output = add_keywords(df_input, df_output)

# disciplines
df_output["disciplines"] = "" #make blank column, we will need to fill in the values

#source_publication
df_output["source_publication"] = "" #make blank column, we will need to fill in the values

# related_data
df_output = add_related_items(df1=df_input, df2=df_output)


# external_rep
df_output = add_repository(df_input, df_output)

# license
df_output = add_license(df_input, df_output)


# funder_info
df_output = add_funders(df1=df_input, df2=df_output)

# source_fulltext_url
df_output = doi_link(df_input, df_output)

# author info (<30)
df_output = add_creators(df1=df_input, df2=df_output)

# Remove entries with the same title, keeping the more recent DOI

# Sort by title and DOI (alphabetically)
df_sorted = df_output.sort_values(by=['title', 'source_fulltext_url'], ascending=[True, True])

# Drop duplicates, keeping the first (which now has the smallest DOI)
df_deduped = df_sorted.drop_duplicates(subset='title', keep='first')

df_out = df_deduped.sort_values(by='source_fulltext_url', ascending=False)

df_out.to_excel('batch-upload/datacite_batch_upload.xlsx', index=False)

df_out



,title,orcid,publication_date,abstract,keywords,disciplines,source_publication,related_data,external_rep,distribution_license,...,author29_lname,author29_institution,author30_fname,author30_mname,author30_lname,author30_institution,author31_fname,author31_mname,author31_lname,author31_institution
0,Additional file 2 of The sustainability of hea...,,2025-04-09,<p>Supplementary Material 2.</p>,"Medicine, Biotechnology, Sociology, FOS: Socio...",,,"<p>Is supplement to: <a href=""https://doi.org/...",<p>figshare</p>,http://creativecommons.org/licenses/by/4.0/,...,,,,,,,,,,
53,Additional file 7 of Identifying differentiati...,"<p>Michał Pikuła <a href=""https://orcid.org/00...",2025-02-12,<p>Supplementary Material 7</p>,"Genetics, FOS: Biological sciences",,,"<p>Is supplement to: <a href=""https://doi.org/...",<p>figshare</p>,http://creativecommons.org/licenses/by/4.0/,...,,,,,,,,,,
54,Additional file 6 of Identifying differentiati...,"<p>Michał Pikuła <a href=""https://orcid.org/00...",2025-02-12,<p>Supplementary Material 6</p>,"Genetics, FOS: Biological sciences",,,"<p>Is supplement to: <a href=""https://doi.org/...",<p>figshare</p>,http://creativecommons.org/licenses/by/4.0/,...,,,,,,,,,,
55,Additional file 5 of Identifying differentiati...,"<p>Michał Pikuła <a href=""https://orcid.org/00...",2025-02-12,<p>Supplementary Material 5</p>,"Genetics, FOS: Biological sciences",,,"<p>Is supplement to: <a href=""https://doi.org/...",<p>figshare</p>,http://creativecommons.org/licenses/by/4.0/,...,,,,,,,,,,
58,Additional file 4 of Identifying differentiati...,"<p>Michał Pikuła <a href=""https://orcid.org/00...",2025-02-12,<p>Supplementary Material 4</p>,"Genetics, FOS: Biological sciences",,,"<p>Is supplement to: <a href=""https://doi.org/...",<p>figshare</p>,http://creativecommons.org/licenses/by/4.0/,...,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16,Cystic fibrosis autoantibody signatures associ...,"<p>Balazs Rada <a href=""https://orcid.org/0000...",2023-08-30,<p>While cystic fibrosis (CF) lung disease is ...,"FOS: Biological sciences, FOS: Biological scie...",,,"<p>Is cited by: <a href=""https://doi.org/10.33...",<p>Dryad</p>,https://creativecommons.org/public-domain/cc0/,...,,,,,,,,,,
14,Data from: Evolutionary history of chimpanzees...,,2010-08-19,<p>Investigations into the evolutionary histor...,"chimpanzee, Pan troglodytes, relaxed molecular...",,,"<p>Is cited by: <a href=""https://doi.org/10.10...",<p>Dryad</p>,https://creativecommons.org/public-domain/cc0/,...,,,,,,,,,,
37,Data from: Female investment in offspring size...,,2019-04-20,<p>The timing of reproduction strongly influen...,"tradeoffs, Reproducibility, Anolis sagrei, par...",,,"<p>Is cited by: <a href=""https://doi.org/10.10...",<p>Dryad</p>,https://creativecommons.org/public-domain/cc0/,...,,,,,,,,,,
44,Market forces determine the distribution of a ...,"<p>Jeffrey Morris <a href=""https://orcid.org/0...",2021-09-09,"<p>Many biological functions are leaky, and or...","FOS: Biological sciences, FOS: Biological scie...",,,"<p>Is derived from: <a href=""https://doi.org/1...",<p>Dryad</p>,https://creativecommons.org/public-domain/cc0/,...,,,,,,,,,,
